<a href="https://colab.research.google.com/github/NicolePT/QuarantineGame-cpp/blob/main/HITO%204/Avances%202%20Algoritmos/Algoritmo_Prim_Price.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Se obtiene las posiciones de los **warehouse points** y de los **delivery points**

In [ ]:
import graphviz as gv
import numpy as np
import pandas as pd
import heapq as hq
import time
import math

url = "https://raw.githubusercontent.com/JorgeLMF/cc42_tf_202014828_201915747_201921559_201811355_201714378/main/DATASET/almacenesXyY.csv"
url2 = "https://raw.githubusercontent.com/JorgeLMF/cc42_tf_202014828_201915747_201921559_201811355_201714378/main/DATASET/puntosentregaXyY.csv"
file_almacenes = pd.read_csv(url).to_numpy()
pts_entrega = pd.read_csv(url2).to_numpy()


warehouse_points=[]
for line in file_almacenes:
  warehouse_points.append((int(line[0]),int(line[1])))

delivery_points=[]
for line in pts_entrega:
  delivery_points.append((int(line[0]),int(line[1])))

manhattan = lambda a,b: abs(a[0]-b[0])+abs(a[1]-b[1])

In [ ]:

def clusters(warehouse_points, delivery_points): # La funcion clusters pone los puntos de entrega al punto de almacenes más cercano  

  cl = []
  for i in range(len(warehouse_points)):
    cl.append([])

  for j in range(len(delivery_points)):
    d = delivery_points[j]
    mn = math.inf
    idmn = 0
    for i in range(len(warehouse_points)):
      w = warehouse_points[i]
      dis = manhattan(d,w) #se obtiene la distancia de cada warehouse_points hacia las los delivery_points
      if(dis < mn):
        mn = dis
        idmn = i
    
    cl[idmn].append(d)
  
  return cl



In [ ]:
cl = clusters(warehouse_points, delivery_points)

In [ ]:
cl[0] ##muestra los deliverypoints agrupados para el warehouse [0]

In [ ]:
def prim(G):
  n = len(G)
  visited = [False]*n
  path = [-1]*n
  cost = [math.inf]*n
  q = [(0, n-1)]
  while q:
    _, u = hq.heappop(q)
    if not visited[u]:
      visited[u] = True
      for v,_, w in G[u]:
        if not visited[v] and w < cost[v]:
          cost[v] = w
          path[v] = u
          hq.heappush(q, (w, v))
  return path, cost

In [ ]:
def create_graph_by_warehousepoint(indexwh,group): 
  '''
  Esta funcion crea el grafo con la lista de relaciones que se le va a dar como parámetro. En este caso, lo que retorna clusters().
  Con esto, vamos a crear un grafo y para evitar que sea muy denso, solo cada nodo va a tener un arco con los nodos que tengan las 5 menores distancias,
  con respecto a ese nodo.
  '''
  n=len(group)+1
  G=[[] for _ in range(n)]
  l=[]

  whp=warehouse_points[indexwh]
  idwhp=n-1
  v=[] ##esta lista sirve para obtener las 20 menores cantidades que forman un nodo, con los otros.
  for j,dp1 in enumerate(group):
    for k,dp2 in enumerate(group):
      if j==k: continue
      ##obtenemos las distancias para cada combinacion
      distance1=manhattan(dp1,dp2)
      distance2=manhattan(whp,dp2)
      distance3=manhattan(whp,dp1)

      l.append((j,dp1,k,dp2,distance1))
      l.append((idwhp,whp,k,dp2,distance2))
      l.append((j,dp1,idwhp,whp,distance3))     
      
      v.append(distance1)
      v.append(distance2)
      v.append(distance3)
  v=list(set(v))
  v=v[:80] ##max unido con nodos de 20 diferentes valores
  '''
  La intención que se realiza en v es la siguiente:
  Por ejemplo:
    En un caso se obtiene que v=[0,0,1,2,3,3,7,7,8,9]
    Solo queremos los 5 menores distancias. Por ello, se realiza:
                                                    v=list(set(v))
                                                    v=v[:5]
  Con esto, no necesariamte cada nodo va a estar relacionado con solo 5 nodos, pues puede haber 3 nodos que coincidan en distancia con el 
  nodo escogido.                                                    
  '''

  for idp1,p1,idp2,p2,distance in l:
    if distance in v and (idp2,p2,distance) not in G[idp1]:
      G[idp1].append((idp2,p2,distance))
      G[idp2].append((idp1,p1,distance))
      #G[p1].append((p2,distance))
      #G[p2].append((p1,distance))
  return G
  

In [ ]:
a=[0,1,2,2,2,3,4,6,7]
a=list(set(a))
a=a[:4]
print(a)

In [ ]:
def main():
  cl = clusters(warehouse_points, delivery_points)
  G_total=[[] for _ in range(len(cl))]
  for i,group in enumerate(cl):
    G=create_graph_by_warehousepoint(i,group)
    path,cost=prim(G)
    G_total[i].append(path)
    G_total[i].append(cost) ##van a estar almacenados en orden del warehousepoint que es el mismo de clusters

  return G_total


In [ ]:
np.matrix(main())